# MATH 210 Introduction to Mathematical Computing

## March 7, 2018

* Linear algebra with SciPy
    * Matrix multiplication
    * Solving linear systems $Ax=b$
    * Matrix row operations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Linear algebra with SciPy

Many linear algebra functions and operations are available in the subpackage `scipy.linalg`. See the [documentation](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html).

In [2]:
import scipy.linalg as la

### Matrix multiplication

Recall, NumPy arrays execute operations elementwise.

In [3]:
A = np.array([[1,2],[3,4]])
print(A)

[[1 2]
 [3 4]]


In [4]:
A*A

array([[ 1,  4],
       [ 9, 16]])

This is array elementwise multiplication. What about matrix multiplication?

In [5]:
A @ A

array([[ 7, 10],
       [15, 22]])

The symbol `@` is used for matrix multiplication of NumPy arrays. This is new as of Python 3.5.

What about matrix powers such as $A^3$? We could just matrix multiplication 3 times:

In [6]:
A @ A @ A

array([[ 37,  54],
       [ 81, 118]])

There is a function `numpy.linalg.matrix_power` which computes matrix powers. Notice this is in the subpackage `numpy.linalg`. We will use `scipy.linalg` for linear algebra operations and functions (except for matrix powers). For more information about the differences between `scipy.linalg` and `numpy.linalg`, see the [SciPy documentation](https://www.scipy.org/scipylib/faq.html#why-both-numpy-linalg-and-scipy-linalg-what-s-the-difference).

In [7]:
from numpy.linalg import matrix_power as mpow

In [8]:
mpow(A,3)

array([[ 37,  54],
       [ 81, 118]])

In [9]:
D = np.array([[2,0],[0,-1]])
print(D)

[[ 2  0]
 [ 0 -1]]


In [10]:
mpow(D,5)

array([[32,  0],
       [ 0, -1]])

### Solving linear systems

A linear system of equations in matrix form is $Ax=b$ where $A$ is a m by n matrix, $x$ is a vector of size n by 1 and $b$ is a vector of size m by 1. In `scipy.linalg`, there is a function called `scipy.linalg.solve` for solving linear systems.

In [11]:
la.solve?

Let's solve a random linear system:

In [12]:
A = np.random.randint(-5,5,(3,3))
print(A)

[[-4  4  2]
 [ 1  4  0]
 [ 3 -4  3]]


In [13]:
b = np.random.randint(-5,5,(3,1))
print(b)

[[-2]
 [ 1]
 [ 1]]


In [14]:
x = la.solve(A,b)
print(x)

[[ 0.56521739]
 [ 0.10869565]
 [-0.08695652]]


Let's verify this is the solution:

In [15]:
A @ x

array([[-2.],
       [ 1.],
       [ 1.]])

### Matrix row operations

The function `scipy.linalg.solve` is a power blackbox. How does it work? The documentation doesn't really say but I suspect it uses the Fortran code in [LAPACK](http://www.netlib.org/lapack/lug/). Anyway, let's write our own code to solve systems. Let's write functions for the usual [row operations using matrix multiplication](https://en.wikipedia.org/wiki/Elementary_matrix#Elementary_row_operations).

Let's write a function called `add_row` which takes input parameters $A$, $k$, $i$ and $j$ and returns the NumPy array resulting from adding $k$ times row $i$ to row $j$.

In [16]:
def add_row(A,k,i,j):
    "Add k times row i to row j in matrix A (using 0 indexing)."
    m = A.shape[0] # The number of rows of A
    E = np.eye(m)
    E[j,i] = k
    return E@A

In [17]:
M = np.array([[1,1],[3,2]])
add_row(M,2,0,1)

array([[ 1.,  1.],
       [ 5.,  4.]])

Let's write a function called `swap_row` which takes input parameters $A$, $i$ and $j$ and returns the matrix resulting from swapping rows $i$ and $j$ in $A$.

In [18]:
def swap_row(A,i,j):
    "Swap rows i and j in matrix A (using 0 indexing)."
    nrows = A.shape[0] # The number of rows in A
    E = np.eye(nrows)
    E[i,i] = 0
    E[j,j] = 0
    E[i,j] = 1
    E[j,i] = 1
    return E@A

In [19]:
M = np.array([[1,1],[3,2]])
swap_row(M,0,1)

array([[ 3.,  2.],
       [ 1.,  1.]])

Let's write a function called `scale_row` which takes input parameters $A$, $k$ and $i$ and returns the matrix resulting from multiplying row $i$ by $k$.

In [20]:
def scale_row(A,k,i):
    "Multiply row i by k in matrix (using 0 indexing)."
    nrows = A.shape[0] # The number of rows in A
    E = np.eye(nrows)
    E[i,i] = k
    return E@A

In [21]:
M = np.array([[1,1],[3,2]])
scale_row(M,3,1)

array([[ 1.,  1.],
       [ 9.,  6.]])

Let's use our array operations to solve $Mx=b$ for $b = [1,-1]^T$

In [22]:
M

array([[1, 1],
       [3, 2]])

In [23]:
b = np.array([[1],[-1]])
print(b)

[[ 1]
 [-1]]


In [24]:
A = np.hstack([M,b])
print(A)

[[ 1  1  1]
 [ 3  2 -1]]


In [25]:
A1 = add_row(A,-3,0,1)
print(A1)

[[ 1.  1.  1.]
 [ 0. -1. -4.]]


In [26]:
A2 = scale_row(A1,-1,1)
print(A2)

[[ 1.  1.  1.]
 [ 0.  1.  4.]]


In [27]:
A3 = add_row(A2,-1,1,0)
print(A3)

[[ 1.  0. -3.]
 [ 0.  1.  4.]]


Our solution is $x = [-3,4]^T$. Let's see if `scipy.linalg.solve` agrees.

In [28]:
x = la.solve(M,b)
print(x)

[[-3.]
 [ 4.]]


Success!